# How to Fetch Data from BigQuery Using GCloud Python API
To familiarize yourself with _bigquery concepts_, and _bigquery export schema_ for GA. You may get information here
1. BigQuery Concepts [https://cloud.google.com/bigquery/what-is-bigquery](https://cloud.google.com/bigquery/what-is-bigquery)
2. BigQuery Export Schema [https://support.google.com/analytics/answer/3437719?hl=en](https://support.google.com/analytics/answer/3437719?hl=en)
3. GCloud Python API [https://googlecloudplatform.github.io/gcloud-python/0.14.0/bigquery-usage.html](https://googlecloudplatform.github.io/gcloud-python/0.14.0/bigquery-usage.html)

### 1. Import necessary module
We need to import bigquery from gcloud and ServiceAccountCredentials from oauth2client.service_account

In [72]:
from gcloud import bigquery
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

### 2. Create service account credentials from json key file
All request to bigquery API must be authenticated using service account or user id. We use service account authentication here to fetch data from bigquery. For the same purpose we need to create credentials object from key file. 

To know more about it visit [https://cloud.google.com/bigquery/authentication](https://cloud.google.com/bigquery/authentication)

In [73]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(filename="keyfile.json")

### 3. Create bigquery client to execute queries, get datasets and tables

#### 3.1 Create Client
To create client we need to pass to mandatory parameter
1. Credentials - This is a oauth2client credentials object that is required to authenticate API call
2. Project - Project ID to which each API call is made

In [74]:
client = bigquery.Client(credentials = credentials, project='project_id_here')

#### 3.2 Get List of Datasets 
Make an API call to get all datasets in the project. Bigquery API call returns results in pages. When there is no more page `next_page_token` would be null, otherwise it is a pointer to next page result. 

In [75]:
datasets, next_page_token = client.list_datasets() # API Call
for dataset in datasets:
    print (dataset.name)

102325651
34831806
6052661
tmp


#### 3.3 Get All Tables in a Particular Dataset

In [76]:
tables, next_page_token = datasets[0].list_tables()
table_name = []
while True:
    for table in tables:
        table_name.append(table.name)
    if next_page_token:
        tables, next_page_token = datasets[0].list_tables(page_token=next_page_token)
    else:
        break

print('\n'.join(table_name[-10:]))

ga_sessions_20160810
ga_sessions_20160811
ga_sessions_20160812
ga_sessions_20160813
ga_sessions_20160814
ga_sessions_20160815
ga_sessions_20160816
ga_sessions_20160817
ga_sessions_20160818
ga_sessions_intraday_20160819


#### 3.4 Fetch Data From BigQuery
To fetch data we need to create a query string which is simply GBQ query. Thereafter we can invoke `client.run_sync_query` or `client.run_async_query` to get data from bigquery. The former runs in blocking/sync mode but the later will run in background and we need to poll the job status for the completion. 

##### TIP - It is suggested that always use maxResults, because in a single response bigquery can't return more than 10 MB of data. 
For more information on this limit visit [https://cloud.google.com/bigquery/docs/data#paging-through-list-results](https://cloud.google.com/bigquery/docs/data#paging-through-list-results)

In [86]:
query = '''
    SELECT 
        IF(totals.newvisits IS NULL, 'Returning User', 'New User') AS USER_TYPE,
        DATE,
        COUNT(totals.visits) SESSIONS,
        AVG(totals.pageviews) AS PAGE_DEPTH
    FROM
        TABLE_DATE_RANGE([34831806.ga_sessions_], TIMESTAMP('2016-07-01'), TIMESTAMP('2016-07-07'))
    GROUP BY
        USER_TYPE, DATE
    ORDER BY DATE, USER_TYPE
'''
query_result = client.run_sync_query(query)
query_result.run()

In [89]:
# Print schema information of the returned result
print (map(lambda x: (x.name, x.field_type), query_result.schema))
# print data in returned response
# Each row is represented by a tuple and entire result is just a python array of tuples
print ('\n'.join(map(str, query_result.rows)))

[(u'USER_TYPE', u'STRING'), (u'DATE', u'STRING'), (u'SESSIONS', u'INTEGER'), (u'PAGE_DEPTH', u'FLOAT')]
(u'New User', u'20160701', 924401, 4.984869956922758)
(u'Returning User', u'20160701', 2906356, 6.286320121678447)
(u'New User', u'20160702', 849042, 5.028440691141656)
(u'Returning User', u'20160702', 2738384, 6.132032899206007)
(u'New User', u'20160703', 896375, 5.432021250366761)
(u'Returning User', u'20160703', 2854690, 6.87257143758003)
(u'New User', u'20160704', 1013768, 5.602692958357789)
(u'Returning User', u'20160704', 3405384, 7.002611140120516)
(u'New User', u'20160705', 1084170, 5.822655595689182)
(u'Returning User', u'20160705', 3612524, 7.460852318612158)
(u'New User', u'20160706', 973246, 6.112778864648923)
(u'Returning User', u'20160706', 3350059, 7.240907500528408)
(u'New User', u'20160707', 975007, 6.1546255669265655)
(u'Returning User', u'20160707', 3305482, 7.255116242481453)
